In [2]:
!pip install -q pyautogen python-dotenv gradio google-generativeai "ag2[gemini]"


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import autogen

In [10]:
import os
import autogen
import gradio as gr

from dotenv import load_dotenv
from IPython.display import display, Markdown
import random  # Used later for unique Gradio outputs
import google.generativeai as genai

In [11]:
# Load environment variables from the .env file
load_dotenv()

# Retrieve API keys from environment variables
#openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")  # Load the Google API key

print("Setup Complete: Libraries installed and API keys loaded.")

Setup Complete: Libraries installed and API keys loaded.


In [12]:


config_list_gemini = [
    {
        "model": "gemini-2.5-flash", 
        "api_key": google_api_key,
        "api_type": "google",
    }
]

# Unified LLM Config for all agents
gemini_llm_config = {
    "config_list": config_list_gemini,
    "temperature": 0.7, # Balanced creativity and precision
    "timeout": 120,
}

In [13]:
# 2. DEFINE PROMPTS (System Messages)
cmo_system_message = """You are the Chief Marketing Officer (CMO). 
Focus on high-level strategy, market positioning, and overall business goals. 
Critique ideas based on budget, feasibility, and brand alignment."""

brand_marketer_system_message = """You are the Brand Marketer. 
Focus on creative campaign concepts, target audience analysis, and messaging hooks. 
Brainstorm distinct campaign concepts outlining core ideas and channels."""

social_media_system_message = """You are the Social Media Strategist for a new sustainable shoe brand. 
Develop creative social media posts, suggest platforms, posting times, and engagement strategies. 
Work with the Brand Marketer and CMO."""

In [14]:
# 3. CREATE AGENTS (All using Gemini)

# User Proxy Agent (Represents you)
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    system_message="A human admin. Interact with the team to provide feedback or approve plans.",
    code_execution_config=False,
    human_input_mode="TERMINATE", # Stops to ask for your input; change to "NEVER" for full automation
)

# Chief Marketing Officer (Powered by Gemini)
cmo_agent = autogen.ConversableAgent(
    name="Chief_Marketing_Officer",
    system_message=cmo_system_message,
    llm_config=gemini_llm_config,
    human_input_mode="NEVER",
)

# Brand Marketer (Powered by Gemini - Replaces OpenAI)
brand_marketer_agent = autogen.ConversableAgent(
    name="Brand_Marketer",
    system_message=brand_marketer_system_message,
    llm_config=gemini_llm_config,
    human_input_mode="NEVER",
)

# Social Media Strategist (Powered by Gemini - Replaces Claude)
social_media_agent = autogen.ConversableAgent(
    name="Social_Media_Strategist",
    system_message=social_media_system_message,
    llm_config=gemini_llm_config,
    human_input_mode="NEVER",
)

In [15]:
# 4. CREATE GROUP CHAT
# The GroupChatManager essentially acts as the moderator and also needs an LLM.
# We assign it Gemini as well.

group_chat = autogen.GroupChat(
    agents=[user_proxy, cmo_agent, brand_marketer_agent, social_media_agent],
    messages=[],
    max_round=12, # Limit the conversation length
)

group_manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=gemini_llm_config # Using Gemini to manage the chat flow
)

In [16]:
# 5. DEFINE THE TASK AND START CHAT
initial_task = """
Context: We're launching a new sustainable shoe line and need campaign ideas.
Input: Our sustainable, futuristic shoe brand needs marketing direction.

Team task:
1. CMO: Outline strategic goals.
2. Brand Marketer: Suggest a campaign hook and core idea.
3. Social Media Strategist: Provide a social media plan based on the hook.

Let's start!
"""

print("--- Starting Group Chat with All-Gemini Team ---")

chat_result = user_proxy.initiate_chat(
    recipient=group_manager,
    message=initial_task,
)

print("--- Conversation Ended ---")

--- Starting Group Chat with All-Gemini Team ---
User_Proxy (to chat_manager):


Context: We're launching a new sustainable shoe line and need campaign ideas.
Input: Our sustainable, futuristic shoe brand needs marketing direction.

Team task:
1. CMO: Outline strategic goals.
2. Brand Marketer: Suggest a campaign hook and core idea.
3. Social Media Strategist: Provide a social media plan based on the hook.

Let's start!


--------------------------------------------------------------------------------

Next speaker: Chief_Marketing_Officer

Chief_Marketing_Officer (to chat_manager):

Alright team, excellent to kick off this exciting new venture. As CMO, my focus is on ensuring this launch isn't just a splash, but a strategic move that establishes us as a leader in a compelling new category. We're not just selling shoes; we're selling a vision for the future of footwear.

Here are our strategic goals for this launch campaign:

1.  **Establish Category Leadership & Brand Authority:**
   

In [19]:
##2. Next agent 

# --- 1. DEFINE SYSTEM MESSAGES (Roles) ---

destination_researcher_prompt = """You are the Destination Research Agent.
Your role is to match traveler preferences with suitable locations.
You have extensive knowledge of global destinations, seasonal considerations, and travel trends.
Responsibilities:
- Suggest destinations based on budget, interests (adventure, relaxation, culture), weather, and time constraints.
- Provide high-level pros and cons for potential locations.
- Hand off to the Logistics Agent once a destination is agreed upon."""

logistics_booking_prompt = """You are the Logistics & Booking Agent.
Your role is to find and plan the best travel arrangements.
Responsibilities:
- Search for realistic flight options, accommodation deals, and local transportation.
- Handle scheduling, price comparisons, and logistics.
- Ensure all travel components (flights, hotels, transfers) work together seamlessly.
- Be realistic about travel times and costs."""

local_experience_prompt = """You are the Local Experience Agent.
Your role is to provide deep local knowledge about the chosen destination.
Responsibilities:
- Recommend authentic restaurants, hidden attractions, and cultural norms.
- Create personalized daily itineraries based on interests and pace.
- Adapt plans for weather or specific local events.
- Suggest unique experiences that tourists often miss."""

In [20]:
# --- 1. CREATE AGENTS ---

# User Proxy (Represents the traveler)
user_proxy = autogen.UserProxyAgent(
    name="Traveler_Admin",
    system_message="A human traveler. Interact with the team to provide preferences, feedback, or approve plans.",
    code_execution_config=False,
    human_input_mode="TERMINATE", 
)

# Destination Research Agent
destination_agent = autogen.ConversableAgent(
    name="Destination_Researcher",
    system_message=destination_researcher_prompt,
    llm_config=gemini_llm_config,
    human_input_mode="NEVER",
)

# Logistics & Booking Agent
logistics_agent = autogen.ConversableAgent(
    name="Logistics_Manager",
    system_message=logistics_booking_prompt,
    llm_config=gemini_llm_config,
    human_input_mode="NEVER",
)

# Local Experience Agent
local_agent = autogen.ConversableAgent(
    name="Local_Guide",
    system_message=local_experience_prompt,
    llm_config=gemini_llm_config,
    human_input_mode="NEVER",
)

In [21]:
# --- 5. GROUP CHAT SETUP ---
group_chat = autogen.GroupChat(
    agents=[user_proxy, destination_agent, logistics_agent, local_agent],
    messages=[],
    max_round=12, # Limits the conversation to 12 turns to save API usage
)

# The Manager uses Gemini to moderate the discussion
group_manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=gemini_llm_config
)

In [22]:
# ---  START THE CHAT ---
# We provide a specific travel scenario to kickstart the agents
initial_task = """
Context: A couple wants to plan a 10-day trip.
Input: We have a budget of $5,000. We love food, hiking, and history. 
We are thinking about traveling in November, either to Japan or Italy.

Team task:
1. Destination Researcher: Compare Japan vs Italy for November based on our interests and budget. Pick a winner.
2. Logistics Manager: Outline a rough budget breakdown (flights/hotels) for the winning choice.
3. Local Guide: Suggest 3 unique "hidden gem" experiences for that location.
"""

print("\n--- Starting Vacation Planning Session ---")
chat_result = user_proxy.initiate_chat(
    recipient=group_manager,
    message=initial_task,
)
print("--- Planning Session Ended ---")


--- Starting Vacation Planning Session ---
Traveler_Admin (to chat_manager):


Context: A couple wants to plan a 10-day trip.
Input: We have a budget of $5,000. We love food, hiking, and history. 
We are thinking about traveling in November, either to Japan or Italy.

Team task:
1. Destination Researcher: Compare Japan vs Italy for November based on our interests and budget. Pick a winner.
2. Logistics Manager: Outline a rough budget breakdown (flights/hotels) for the winning choice.
3. Local Guide: Suggest 3 unique "hidden gem" experiences for that location.


--------------------------------------------------------------------------------

Next speaker: Destination_Researcher

Destination_Researcher (to chat_manager):

Hello! I'm happy to help you explore your options for a 10-day trip. Both Japan and Italy are fantastic choices for food, hiking, and history, and November can be a beautiful time to visit. Let's break down how they stack up against your preferences:

### Japan vs. It